In [1]:
%matplotlib inline

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

from tensorflow.python.client import device_lib

print(tf.version.VERSION)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

2.9.1
Num GPUs Available:  1


2022-08-31 19:54:55.376559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-31 19:54:55.403224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-31 19:54:55.403552: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
(train_images, train_labels), (
    test_images,
    test_labels,
) = keras.datasets.mnist.load_data()

size = 1000
image_size = 28 * 28

train_labels = train_labels[:size]
test_labels = test_labels[:size]

train_images = train_images[:size].reshape(-1, image_size) / 255.0
test_images = test_images[:size].reshape(-1, image_size) / 255.0

11490434/11490434 [==============================] - 2s 0us/step


In [4]:
def create_model():
    model = keras.models.Sequential(
        [
            keras.layers.Dense(512, activation="relu", input_shape=(image_size,)),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(256, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()],
    )
    return model

model=create_model()

model.summary()

2022-08-31 20:02:45.011736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-31 20:02:45.013905: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-31 20:02:45.014372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-31 20:02:45.014710: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1
)

model.fit(train_images,train_labels,epochs=10,validation_data=(test_images,test_labels),callbacks=[cp_callback])

Epoch 1/10


2022-08-31 20:05:24.844834: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


26/32 [=======================>......] - ETA: 0s - loss: 1.8124 - sparse_categorical_accuracy: 0.4062
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 4s 14ms/step - loss: 1.7408 - sparse_categorical_accuracy: 0.4300 - val_loss: 0.9616 - val_sparse_categorical_accuracy: 0.7350
Epoch 2/10
25/32 [======================>.......] - ETA: 0s - loss: 0.8125 - sparse_categorical_accuracy: 0.7250
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.7925 - sparse_categorical_accuracy: 0.7400 - val_loss: 0.6007 - val_sparse_categorical_accuracy: 0.8110
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.5677 - sparse_categorical_accuracy: 0.8214
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.5641 - sparse_categorical_accuracy: 0.8250 - val_loss: 0.5150 - val_sparse_categorical_accuracy: 0.8380
Epoch 4/10
27/32 [===================

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [8]:
model=create_model()

loss,acc=model.evaluate(test_images,test_labels,verbose=2)
print("トレーニング前, acc:{:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3192 - sparse_categorical_accuracy: 0.0990 - 154ms/epoch - 5ms/step
トレーニング前, acc: 9.90%


In [9]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("トレーニング後, acc:{:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.3917 - sparse_categorical_accuracy: 0.8760 - 93ms/epoch - 3ms/step
トレーニング後, acc:87.60%


In [13]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 64

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5 * batch_size,
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=100,
    batch_size=batch_size,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0,
)


Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 60: saving model to training_2/cp-0060.ckpt

Epoch 80: saving model to training_2/cp-0080.ckpt

Epoch 100: saving model to training_2/cp-0100.ckpt


In [15]:
os.listdir(checkpoint_dir)

['cp-0060.ckpt.data-00000-of-00001',
 'cp-0100.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'checkpoint',
 'cp-0060.ckpt.index',
 'cp-0100.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0080.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0080.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index']

In [16]:
latest=tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0100.ckpt'

In [17]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.6188 - sparse_categorical_accuracy: 0.8820 - 147ms/epoch - 5ms/step
Restored model, accuracy: 88.20%


In [18]:
model.save_weights('./checkpoints/my_checkpoint')

model=create_model()

model.load_weights('./checkpoints/my_checkpoint')
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.6188 - sparse_categorical_accuracy: 0.8820 - 154ms/epoch - 5ms/step
Restored model, accuracy: 88.20%


In [19]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=100)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/100
32/32 [==============================] - 0s 6ms/step - loss: 1.7116 - sparse_categorical_accuracy: 0.4300
Epoch 2/100
32/32 [==============================] - 0s 7ms/step - loss: 0.8258 - sparse_categorical_accuracy: 0.7330
Epoch 3/100
32/32 [==============================] - 0s 5ms/step - loss: 0.6023 - sparse_categorical_accuracy: 0.8110
Epoch 4/100
32/32 [==============================] - 0s 4ms/step - loss: 0.4182 - sparse_categorical_accuracy: 0.8740
Epoch 5/100
32/32 [==============================] - 0s 4ms/step - loss: 0.3449 - sparse_categorical_accuracy: 0.8990
Epoch 6/100
32/32 [==============================] - 0s 4ms/step - loss: 0.2594 - sparse_categorical_accuracy: 0.9290
Epoch 7/100
32/32 [==============================] - 0s 4ms/step - loss: 0.2057 - sparse_categorical_accuracy: 0.9330
Epoch 8/100
32/32 [==============================] - 0s 4ms/step - loss: 0.1805 - sparse_categorical_accuracy: 0.9460
Epoch 9/100
32/32 [==============================] - 0s 

In [20]:
!ls saved_model

my_model


In [21]:
!ls saved_model/my_model

assets	keras_metadata.pb  saved_model.pb  variables


In [22]:
new_model=tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 512)               401920    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
